TREINAMENTO DO MODELO DE DETECÇÃO DE LETRAS EM LIBAS

In [90]:
# IMPORT
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf

In [91]:
# Parâmetros
IMG_SIZE = 32
BATCH_SIZE = 32
EPOCHS = 50
DATASET_DIR = "../dataset"

# Data Augmentation + Normalização
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    horizontal_flip=False
)

# Dados de treino
train_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)


# Dados de validação
val_generator = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Mapeamento das classes (letras)
class_indices = train_generator.class_indices
print("Mapeamento das classes:", class_indices)

Found 5919 images belonging to 21 classes.
Found 1472 images belonging to 21 classes.
Mapeamento das classes: {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'i': 7, 'l': 8, 'm': 9, 'n': 10, 'o': 11, 'p': 12, 'q': 13, 'r': 14, 's': 15, 't': 16, 'u': 17, 'v': 18, 'w': 19, 'y': 20}


In [92]:
# Salvar mapeamento para uso posterior
import json
with open('../models/class_indices.json', 'w') as f:
    json.dump(class_indices, f)

In [93]:
# Modelo CNN aprimorado
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 21)             │         2,709 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161,621 (631.33 KB)

 Trainable params: 161,621 (631.33 KB)

 Non-trainable params: 0 (0.00 B)

In [94]:
#Hiperparâmetros
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=hp.Choice('conv_1_filter', values=[32, 64, 128]),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(
        units=hp.Int('dense_units', min_value=64, max_value=256, step=32),
        activation='relu'
    ))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1)))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

num_classes = train_generator.num_classes

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='tuning_results',
    project_name='asl_libras'
)

tuner.search(train_generator, epochs=10, validation_data=val_generator)

# Pega o melhor modelo
best_model = tuner.get_best_models(num_models=1)[0]

Reloading Tuner from tuning_results\asl_libras\tuner0.json


In [95]:
# Treinamento com early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

checkpoint = ModelCheckpoint(
    filepath='../models/best_model.h5',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

history = best_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint]
)

# Plot dos resultados
plt.plot(history.history['accuracy'], label='Acurácia Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.show()

Epoch 1/50
 36/185 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.9542 - loss: 0.1457

KeyboardInterrupt: 

In [98]:
#Cross-validation
X, y = [], []
for batch_x, batch_y in train_generator:
    X.extend(batch_x)
    y.extend(batch_y)
    if len(X) >= train_generator.samples:
        break

X = np.array(X)
y = np.array(y)

# Cross-validation com KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold = 1

for train_index, val_index in kf.split(X):
    print(f"\n🔁 Training Fold {fold}")
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = best_model
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        callbacks=[early_stop, checkpoint]
    )
    
    final_loss, final_accuracy = model.evaluate(X_val, y_val)
    print(f"Fold {fold} - Val Loss: {final_loss:.4f}, Val Accuracy: {final_accuracy:.4f}")
    fold += 1



🔁 Training Fold 1
Epoch 1/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9545 - loss: 0.1431
Epoch 1: val_loss did not improve from 0.02086
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9544 - loss: 0.1431 - val_accuracy: 0.9755 - val_loss: 0.0636
Epoch 2/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9764 - loss: 0.0751
Epoch 2: val_loss did not improve from 0.02086
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9765 - loss: 0.0750 - val_accuracy: 0.9831 - val_loss: 0.0739
Epoch 3/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9891 - loss: 0.0347
Epoch 3: val_loss did not improve from 0.02086
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9891 - loss: 0.0348 - val_accuracy: 0.9789 - val_loss: 0.0671
Epoch 4/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9927 - loss: 0.0242
Epoch 4: val_loss did not improve from 0.02086
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9927 - loss: 0.0242 - val_accuracy:

148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9812 - loss: 0.0674 - val_accuracy: 0.9966 - val_loss: 0.0130
Epoch 2/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9815 - loss: 0.0531
Epoch 2: val_loss did not improve from 0.01297
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.9815 - loss: 0.0529 - val_accuracy: 0.9932 - val_loss: 0.0222
Epoch 3/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9911 - loss: 0.0312
Epoch 3: val_loss did not improve from 0.01297
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9911 - loss: 0.0314 - val_accuracy: 0.9958 - val_loss: 0.0173
Epoch 4/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9908 - loss: 0.0280
Epoch 4: val_loss did not improve from 0.01297
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9908 - loss: 0.0280 - val_accuracy: 0.9890 - val_loss: 0.0380
Epoch 5/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9943 - loss: 0.0214
Epoch 5: val_loss did not improve from 

148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9861 - loss: 0.0527 - val_accuracy: 0.9983 - val_loss: 0.0086
Epoch 2/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9910 - loss: 0.0319
Epoch 2: val_loss improved from 0.00859 to 0.00600, saving model to ../models/best_model.h5


148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9910 - loss: 0.0319 - val_accuracy: 1.0000 - val_loss: 0.0060
Epoch 3/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9906 - loss: 0.0269
Epoch 3: val_loss did not improve from 0.00600
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9906 - loss: 0.0270 - val_accuracy: 0.9890 - val_loss: 0.0437
Epoch 4/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9881 - loss: 0.0371
Epoch 4: val_loss did not improve from 0.00600
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9881 - loss: 0.0371 - val_accuracy: 0.9958 - val_loss: 0.0194
Epoch 5/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9938 - loss: 0.0171
Epoch 5: val_loss did not improve from 0.00600
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9937 - loss: 0.0172 - val_accuracy: 0.9958 - val_loss: 0.0150
Epoch 6/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9923 - loss: 0.0282
Epoch 6: val_loss did not improve from 

148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9895 - loss: 0.0313 - val_accuracy: 0.9983 - val_loss: 0.0060
Epoch 5/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9961 - loss: 0.0146
Epoch 5: val_loss did not improve from 0.00596
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9961 - loss: 0.0147 - val_accuracy: 0.9966 - val_loss: 0.0150
Epoch 6/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9898 - loss: 0.0280
Epoch 6: val_loss did not improve from 0.00596
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9898 - loss: 0.0281 - val_accuracy: 0.9958 - val_loss: 0.0139
Epoch 7/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9951 - loss: 0.0165
Epoch 7: val_loss did not improve from 0.00596
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9951 - loss: 0.0165 - val_accuracy: 0.9949 - val_loss: 0.0149
Epoch 8/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9961 - loss: 0.0129
Epoch 8: val_loss did not improve from 

148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9958 - loss: 0.0191 - val_accuracy: 0.9992 - val_loss: 0.0045
Epoch 7/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9943 - loss: 0.0162
Epoch 7: val_loss did not improve from 0.00450
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9943 - loss: 0.0162 - val_accuracy: 0.9992 - val_loss: 0.0055
Epoch 8/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.9984 - loss: 0.0065
Epoch 8: val_loss improved from 0.00450 to 0.00417, saving model to ../models/best_model.h5


148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9984 - loss: 0.0065 - val_accuracy: 0.9983 - val_loss: 0.0042
Epoch 9/50
148/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9955 - loss: 0.0134
Epoch 9: val_loss did not improve from 0.00417
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9954 - loss: 0.0135 - val_accuracy: 0.9882 - val_loss: 0.0289
Epoch 10/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9884 - loss: 0.0412
Epoch 10: val_loss did not improve from 0.00417
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.9884 - loss: 0.0411 - val_accuracy: 0.9899 - val_loss: 0.0469
Epoch 11/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9915 - loss: 0.0272
Epoch 11: val_loss did not improve from 0.00417
148/148 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.9915 - loss: 0.0272 - val_accuracy: 0.9949 - val_loss: 0.0141
Epoch 12/50
147/148 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9910 - loss: 0.0294
Epoch 12: val_loss did not improve

In [99]:
# Salva o modelo treinado
best_model.save('../models/asl_gesture_model2.h5')
print("Modelo salvo como 'asl_gesture_model2.h5'")

Modelo salvo como 'asl_gesture_model2.h5'
